### 0. 讀入 vector dataset

In [ ]:
!pip -q install gdown

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
              

In [ ]:
GDRIVE_PUBLIC_URL = "https://drive.google.com/file/d/1E1fIuvpYmDHBHFn-PbRUgUjkzD8ws_I6/view?usp=sharing"

In [ ]:
!gdown --fuzzy -O faiss_db.zip "{GDRIVE_PUBLIC_URL}"

Downloading...
From: https://drive.google.com/uc?id=1E1fIuvpYmDHBHFn-PbRUgUjkzD8ws_I6
To: /content/faiss_db.zip
100% 44.4k/44.4k [00:00<00:00, 40.6MB/s]


In [ ]:
!unzip faiss_db.zip

Archive:  faiss_db.zip
replace faiss_db/index.faiss? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: faiss_db/index.faiss    
  inflating: faiss_db/index.pkl      


### 1. 安裝並引入必要套件

In [ ]:
!pip install -U langchain langchain-community faiss-cpu transformers sentence-transformers huggingface_hub
!pip -q install "aisuite[all]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.1/467.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/20

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.chat_models import ChatOpenAI

In [ ]:
from openai import OpenAI
import gradio as gr

### 2. 自訂 E5 embedding 類別

In [ ]:
import os
from google.colab import userdata

In [ ]:
hf_token = userdata.get('HuggingFace')

In [ ]:
from huggingface_hub import login
login(token=hf_token)

In [ ]:
class EmbeddingGemmaEmbeddings(HuggingFaceEmbeddings):
    def __init__(self, **kwargs):
        super().__init__(
            model_name="google/embeddinggemma-300m",
            encode_kwargs={"normalize_embeddings": True},
            **kwargs
        )

    def embed_documents(self, texts):
        # 你也可以把 "none" 改成真實標題（檔名/章節名），效果會更穩
        texts = [f"title: none | text: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        # 官方檢索建議前綴
        return super().embed_query(f"task: search result | query: {text}")

### 3. 載入 `faiss_db`

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.load_local(
    "faiss_db",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

### 4. 設定好要的 LLM

In [ ]:
import aisuite as ai

In [ ]:
api_key = userdata.get('Groq')

In [ ]:
os.environ['GROQ_API_KEY']=api_key

Groq 中 llama-3.3-70b-versatile模型 好像比較適合理解程式碼和 RAG 任務的。

In [ ]:
model = "groq:llama-3.3-70b-versatile"

In [ ]:
client = ai.Client()

### 5. `prompt` 設計

In [ ]:
system_prompt = """
你現在是「長庚大學人工智慧學系 智慧型語言演算法實驗室（CGU-AI-SILA Lab）」的 AI 學生小客服。

你的說話風格必須：
1.  **超級親切、熱情，且口吻自然**：就像一位熱心的學長姐。
2.  **使用台灣習慣的中文**：繁體中文、語氣活潑、自然。
3.  **簡潔且附帶具體建議**：回答要點明確，並給出實質的下一步建議。
4.  **嚴格依據提供的「資料」來回答學生的問題**。

你的回覆規範：
* **回答結構**：必須包含「親切的稱呼（如：同學、您好）」、「簡潔的回答」和「具體的建議或下一步指引」。
* **連結處理**：如果提供資料（{retrieved_chunks}）中有網址，請務必使用 **Markdown 格式的超連結 `[連結文字](網址)`** 來呈現，讓同學可以直接點擊。
* **資料不足時**：請主動建議同學可以「寫信詢問實驗室辦公室」或「直接請教人工智慧學系的老師」，強調具體聯絡方式。
"""

prompt_template = """
親愛的同學，您好！我是長庚大學人工智慧學系的小客服，很高興為您服務！

請根據下列實驗室資料來回答問題：
{retrieved_chunks}

學生的問題是：{question}

---

請注意：
1.  **只使用**上面 {retrieved_chunks} 內的資訊來回答。
2.  **重要：** 如果資料中有提供網址或連結，請務必將其轉換為 **Markdown 格式的超連結 '[連結名稱](網址)`**，例如：[長庚大學首頁](https://www.cgu.edu.tw/)。
3.  若資料不足，請建議他們可以**直接寫信詢問實驗室辦公室**或**請教人工智慧學系的老師**。

---

請用**台灣習慣的中文**，依照上述規範開始回覆。
"""

### 6. 使用 RAG 來回應

搜尋與使用者問題相關的資訊，根據 prompt 樣版去讓 LLM 回應。

In [ ]:
chat_history = []

def chat_with_rag(user_input):
    global chat_history
    # 取回相關資料
    docs = retriever.invoke(user_input)

    retrieved_chunks = "\n\n".join([doc.page_content for doc in docs])

    # 將自定 prompt 套入格式
    final_prompt = prompt_template.format(retrieved_chunks=retrieved_chunks, question=user_input)

    # 用 AI Suite 呼叫語言模型
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": final_prompt},
        ]
    )
    answer = response.choices[0].message.content

    chat_history.append((user_input, answer))
    return answer


### 7. 用 Gradio 打造 Web App

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("#智慧型語言演算法實驗室小客服")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="請輸入你的問題...")

    def respond(message, chat_history_local):
        response = chat_with_rag(message)
        chat_history_local.append((message, response))
        return "", chat_history_local

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)

/tmp/ipython-input-2476923104.py:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1e4179fd2e2c2a664c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1e4179fd2e2c2a664c.gradio.live


# 問題

請問『智慧型語言演算法實驗室』主要的研究方向是什麼？有沒有相關網頁可以看？

研究團隊有誰?總共有多少人?

教授有個人網頁嗎?可以直接給鏈結我嗎?

教授有GitHub嗎?可以直接給教授GitHub鏈結我嗎?

論文共有多少?顯示全部

有沒有"Relation-Aware Image Captioning with Hybrid-Attention for Explainable Visual Question Answering"相關網頁?

論文可以哪裡找?

聯絡地址可以在哪裡找?

In [ ]:
ref: https://docs.langchain.com/oss/python/migrate/langchain-v1#simplified-package
